In [ ]:
# Data Validation Notebook for Healthcare Insurance MLOps
# This notebook validates prerequisite data before running batch inference

import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Get parameters
dbutils.widgets.text("catalog", "juan_dev", "Unity Catalog name")
dbutils.widgets.text("schema", "healthcare_data", "Schema name")
dbutils.widgets.text("validation_date", "", "Validation date (YYYY-MM-DD)")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
validation_date = dbutils.widgets.get("validation_date")

print(f"Validating data for {catalog}.{schema} on {validation_date}")

In [ ]:
# Validate prerequisite tables exist
required_tables = [
    f"{catalog}.{schema}.silver_patients",
    f"{catalog}.{schema}.dim_patients"
]

validation_results = []

for table_name in required_tables:
    try:
        df = spark.table(table_name)
        row_count = df.count()
        validation_results.append({
            "table": table_name,
            "status": "SUCCESS",
            "row_count": row_count,
            "message": f"Table exists with {row_count:,} rows"
        })
        print(f"✅ {table_name}: {row_count:,} rows")
    except Exception as e:
        validation_results.append({
            "table": table_name,
            "status": "FAILED",
            "row_count": 0,
            "message": str(e)
        })
        print(f"❌ {table_name}: {str(e)}")

In [ ]:
# Check data freshness for dim_patients
try:
    dim_patients = spark.table(f"{catalog}.{schema}.dim_patients")
    
    # Check for current records
    current_records = dim_patients.filter(col("is_current_record") == True).count()
    total_records = dim_patients.count()
    
    print(f"✅ Current records: {current_records:,} out of {total_records:,} total")
    
    # Check required columns
    required_columns = [
        "patient_natural_key", "patient_sex", "patient_region", 
        "patient_age_category", "patient_bmi_category", "patient_smoking_status",
        "is_current_record"
    ]
    
    missing_columns = [col for col in required_columns if col not in dim_patients.columns]
    
    if missing_columns:
        print(f"❌ Missing required columns: {missing_columns}")
        dbutils.notebook.exit(json.dumps({"status": "FAILED", "message": f"Missing columns: {missing_columns}"}))
    else:
        print(f"✅ All required columns present")
        
except Exception as e:
    print(f"❌ Data validation failed: {str(e)}")
    dbutils.notebook.exit(json.dumps({"status": "FAILED", "message": str(e)}))

In [ ]:
# Final validation summary
failed_validations = [r for r in validation_results if r["status"] == "FAILED"]

if failed_validations:
    print(f"❌ Validation failed: {len(failed_validations)} issues found")
    for failure in failed_validations:
        print(f"  - {failure['table']}: {failure['message']}")
    dbutils.notebook.exit(json.dumps({"status": "FAILED", "failed_tables": len(failed_validations)}))
else:
    print(f"✅ All validations passed - ready for batch inference")
    dbutils.notebook.exit(json.dumps({"status": "SUCCESS", "validated_tables": len(validation_results)}))